In [1]:
from getpass import getpass
import os

from langchain.document_loaders.unstructured import UnstructuredFileLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma, Pinecone
import tqdm.auto
import pinecone

In [2]:
os.environ['OPENAI_API_KEY'] = getpass()

In [3]:
os.environ['PINECONE_API_KEY'] = getpass()

In [5]:
loader = UnstructuredFileLoader('rules.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [6]:
embeddings = OpenAIEmbeddings()

In [7]:
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment='us-west4-gcp-free'
)
index_name = 'lc0'

In [17]:
docsearch = Pinecone.from_texts(
    [text.page_content for text in texts],
    embeddings,
    index_name=index_name
)

In [27]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [38]:
chain = load_qa_chain(llm=OpenAI(temperature=0))

In [43]:
query = 'Что это за текст?'
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Это примечание ИЗПИ, которое содержит сведения о чистом процентном доходе после дня его первого официального опубликования.'